In [1]:
# Importamos las bibliotecas necesarias y el dataset previamente limpiado

import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from sklearn.model_selection import StratifiedKFold

from itertools import combinations
import sklearn.metrics as metrics

In [2]:
data_gym = pd.read_excel('DatosParaModelo.xlsx')

# Logistic Regression

In [3]:
def Logistic_Regression (DataFrame, Splits):

    lista_AUC_kfolds = []

    Modelo_reg_log = LogisticRegression(penalty=None, max_iter=3000)

    # Se configuran las variables de entrada y salida, la lista de identificadores de clientes y se prepara las divisiones del dataset con estratificación
    X = DataFrame.drop(columns = ['IdPersona', 'Baja', 'Mes'])
    y = DataFrame['Baja']
    Id_Clientes = DataFrame['IdPersona']
    kfold = StratifiedKFold(n_splits=Splits, shuffle=True, random_state=1)

    # Bucle para iterar sobre los diferentes folds separando entre train y test con divisiones estratificadas según Ids de clientes
    for Train_data, Test_data in kfold.split(X, y, groups = Id_Clientes):
        X_train, X_test = X.iloc[Train_data], X.iloc[Test_data]
        y_train, y_test = y.iloc[Train_data], y.iloc[Test_data]

        # Entrenamiento del modelo y cálculo del AUC_PR sobre el conjunto de test
        Modelo_reg_log.fit(X_train, y_train)
        Predicciones_reg = Modelo_reg_log.predict_proba(X_test)
        AUC_ROC = roc_auc_score(y_test, Predicciones_reg[:, 1])
        lista_AUC_kfolds.append(AUC_ROC)

    # Resultado
    Mean_AUC = np.mean(lista_AUC_kfolds)
    return print(f'El AUC del Modelo de Regresión Logística es: {Mean_AUC}')

# Random Forest

In [4]:
def Random_Forest(Profundidades, Estimadores, DataFrame, Splits):

  lista_parametros = []
  lista_AUC = []

  for Depth in Profundidades:
    for n_estimators in Estimadores:
      lista_AUC_kfolds = []
      
      # Se configuran las variables de entrada y salida, la lista de identificadores de clientes y se prepara las divisiones del dataset con estratificación
      X = DataFrame.drop(columns = ['IdPersona', 'Baja', 'Mes'])
      y = DataFrame['Baja']
      Id_Clientes = DataFrame['IdPersona']
      kfold = StratifiedKFold(n_splits=Splits, shuffle=True, random_state=1)

      # Bucle para iterar sobre los diferentes folds separando entre train y test con divisiones estratificadas según Ids de clientes   
      for Train_data, Test_data in kfold.split(X, y, Id_Clientes):
        X_train, X_test = X.iloc[Train_data], X.iloc[Test_data]
        y_train, y_test = y.iloc[Train_data], y.iloc[Test_data]    
    
        # Entrenamiento del modelo y cálculo del AUC_PR sobre el conjunto de test
        Modelo_random_forest = RandomForestClassifier(max_depth=Depth, n_estimators=n_estimators, random_state=1).fit(X_train, y_train)
        Predicciones_rndm_frst = Modelo_random_forest.predict_proba(X_test)
        AUC_ROC = roc_auc_score(y_test, Predicciones_rndm_frst[:, 1])
        lista_AUC_kfolds.append(AUC_ROC)

      # Promediado de los resultados de los diferentes folds y agregado de resultado en las listas
      Mean_AUC = np.mean(lista_AUC_kfolds)
      lista_AUC.append(Mean_AUC)    
      lista_parametros.append([Depth, n_estimators])

  # Resultados
  AUC_max_value = np.argmax(lista_AUC)
  return print(f'El AUC del modelo Random Forest es {lista_AUC[AUC_max_value]}, para un nivel de profundidad de {lista_parametros[AUC_max_value][0]} y consta de {lista_parametros[AUC_max_value][1]} árboles de decisión')

# Gradient Boosting Classifier

In [5]:
def Gradient_Boosting (Profundidades, Estimadores, DataFrame, Splits):

    lista_parametros = []
    lista_AUC = []

    for Depth in Profundidades:
        for n_estimators in Estimadores:
            lista_AUC_kfolds = []

            # Se configuran las variables de entrada y salida, la lista de identificadores de clientes y se prepara las divisiones del dataset con estratificación
            X = DataFrame.drop(columns = ['IdPersona', 'Baja', 'Mes'])
            y = DataFrame['Baja']
            Id_Clientes = DataFrame['IdPersona']
            kfold = StratifiedKFold(n_splits=Splits, shuffle=True, random_state=1)
            
            # Bucle para iterar sobre los diferentes folds separando entre train y test con divisiones estratificadas según Ids de clientes
            for Train_data, Test_data in kfold.split(X, y, groups = Id_Clientes):
                X_train, X_test = X.iloc[Train_data], X.iloc[Test_data]
                y_train, y_test = y.iloc[Train_data], y.iloc[Test_data]

                # Entrenamiento del modelo y cálculo del AUC_PR sobre el conjunto de test
                Modelo_xgboost = xgb.XGBClassifier(n_estimators=n_estimators, max_depth=Depth, objective='binary:logistic').fit(X_train, y_train)
                Predicciones_xgb = Modelo_xgboost.predict_proba(X_test)
                AUC_ROC = roc_auc_score(y_test, Predicciones_xgb[:, 1])
                lista_AUC_kfolds.append(AUC_ROC)

            # Promediado de los resultados de los diferentes folds y agregado de resultado en las listas
            Mean_AUC = np.mean(lista_AUC_kfolds)
            lista_AUC.append(Mean_AUC)
            lista_parametros.append([Depth, n_estimators])

    # Resultados
    AUC_max_value = np.argmax(lista_AUC)
    return print(f'El AUC del modelo XGBoost es {lista_AUC[AUC_max_value]}, para un nivel de profundidad de {lista_parametros[AUC_max_value][0]} y consta de {lista_parametros[AUC_max_value][1]} árboles de decisión')

# Variables de los Modelos

In [6]:
Splits = 50
Profundidades_xgb = [2, 3, 4, 5, 6, 7, 8, 9, 10]
Estimadores_xgb = [20, 40, 60, 80, 100, 120, 140, 160, 180, 200]
Profundidades_forest = [10, 11, 12, 13, 14, 15]
Estimadores_forest = [150, 200, 250, 300]

# Ejecución de Modelos

In [7]:
Logistic_Regression(data_gym, Splits)

El AUC del Modelo de Regresión Logística es: 0.8354133811049952


In [8]:
Gradient_Boosting(Profundidades_xgb, Estimadores_xgb, data_gym, Splits)

El AUC del modelo XGBoost es 0.8629916358924864, para un nivel de profundidad de 5 y consta de 40 árboles de decisión


In [9]:
Random_Forest(Profundidades_forest, Estimadores_forest, data_gym, Splits)

El AUC del modelo Random Forest es 0.8665523290159848, para un nivel de profundidad de 14 y consta de 300 árboles de decisión
